# 3. Edit Distance, Assembly, and Overlaps

This notebook explores the theory and application of edit distance and sequence overlaps to answer the following questions (my answers are also provided):

- Question 1: What is the edit distance of the best match between pattern `GCTGATCGATCGTACG` and the excerpt of human chromosome 1?  (Don't consider reverse complements.)
    - 3

- Question 2: What is the edit distance of the best match between pattern 
`GATTTACCAGATTGAG` and the excerpt of human chromosome 1?  (Don't consider reverse complements.)
    - 2

- Question 3: Create an overlap graph between the provided fastq file and the exerpt of human chromosome 1. How many edges are in the graph? In other words, how many distinct pairs of reads overlap?
    - 904745

- Question 4: Picture the overlap graph corresponding to the overlaps computed for the previous question. How many nodes in this graph have at least one outgoing edge? In other words, how many reads have a suffix involved in an overlap?
    - 7161


In [96]:
# Read genome from FASTA file in folder (chr1.GRCh38.excerpt.fasta) and prepare pattern matching variables

# Set FASTA filename
filename = 'chr1.GRCh38.excerpt.fasta'

def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

# Set text 't' to the genome from file
t = readGenome(filename)

# Specify shorter text 't' for testing
# t = 'TATTGGCTATACGGTT'

# Set pattern 'p' to desired string to match
p = 'GCTGATCGATCGTACG'

In [97]:
# Read sequences and qualities from imported FASTQ file

# Set FASTQ filename
filename = 'ERR266411_1.for_asm.fastq'

def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename, 'r') as fq:
        while True:
            fq.readline()  # skip name line
            seq = fq.readline().rstrip()  # read base sequence
            fq.readline()  # skip placeholder line
            qual = fq.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

sequences, qualities = readFastq(filename)

In [98]:
pattern_len = len(p)
min_dist = None

def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]

for i in range(len(t) - pattern_len + 1):
    substring = t[i:i+pattern_len]
    dist = editDistance(p, substring)
    if (min_dist is None) or (dist < min_dist):
        min_dist = dist

print(min_dist)

3


In [13]:
from collections import defaultdict
import urllib.request

# Download FASTQ file
url = "https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq"
filename = "ERR266411_1.for_asm.fastq"
urllib.request.urlretrieve(url, filename)

def read_fastq(fname):
    sequences = []
    with open(fname, 'r') as fh:
        while True:
            fh.readline()  # skip header
            seq = fh.readline().strip()
            fh.readline()  # skip +
            fh.readline()  # skip quality
            if not seq:
                break
            sequences.append(seq)
    return sequences

reads = read_fastq(filename)
print("Total reads loaded:", len(reads))

k = 30

# Build prefix index: map prefix (first k bases) -> set of read indices
prefix_index = defaultdict(set)
for i, read in enumerate(reads):
    prefix_index[read[:k]].add(i)

def overlap(a, b, min_length):
    """Return length of longest suffix of a matching prefix of b (≥ min_length)"""
    max_olen = 0
    start = 0
    while True:
        start = a.find(b[:min_length], start)
        if start == -1:
            break
        olen = len(a) - start
        if olen >= min_length and b.startswith(a[start:]):
            if olen > max_olen:
                max_olen = olen
        start += 1
    return max_olen

seen = {}
outgoing = set()

for i, read in enumerate(reads):
    read_len = len(read)
    # Try all suffixes length >= k
    for start in range(read_len - k, -1, -1):
        suffix = read[start:]
        if len(suffix) < k:
            continue
        prefix = suffix[:k]
        candidates = prefix_index.get(prefix, set())
        for j in candidates:
            if i == j:
                continue
            olen = overlap(read, reads[j], min_length=k)
            if olen >= k:
                # Update with longest overlap found
                if (i, j) not in seen or olen > seen[(i, j)]:
                    seen[(i, j)] = olen
                    outgoing.add(i)

print("Total overlaps found:", len(seen))
print("Reads with at least one outgoing edge:", len(outgoing))


Total reads loaded: 10000
Total overlaps found: 904746
Reads with at least one outgoing edge: 7161
